# Multinomial Logistic Regression

In this script we use multinomial logistic regression to predict the handwritten digits of the MNIST dataset.

In [ ]:
# !pip install geemap
from geemap import ml

In [ ]:
with open("use_files/landsatDataNP_import.csv", "r") as csv_file:
  data = csv_file.read()

with open("use_files/landsatDataNP.csv", "w") as csv_file:
  csv_file.write(data)

In [ ]:
import pandas as pd
land = pd.read_csv("use_files/landsatDataNP.csv")
land.head()

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# first_row = ['ID', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'NDWI', 'NDVI', 'MNDWI', 'NDBI', 'elevation', 'LULC']

main_data = pd.read_csv('use_files/landsatDataNP.csv') 
# main_data = pd.read_csv('/content/landsatDataNP.csv', names = first_row) 
band_df = pd.DataFrame(main_data)
# band_df.Class = band_df.Class - 1

In [ ]:
band_df.head()

In [ ]:
# band_df["B1"] = band_df["B1"] .div(20000)
# band_df["B2"] = band_df["B2"].div(20000)
# band_df["B3"] = band_df["B3"].div(20000)
# band_df["B4"] = band_df["B4"].div(20000)
# band_df["B5"] = band_df["B5"].div(20000)
# band_df["B6"] = band_df["B6"].div(20000)
# band_df["B7"] = band_df["B7"].div(20000)
# band_df["elevation"] = band_df["elevation"].div(10000)

band_df["B1"] = band_df["B1"] 
band_df["B2"] = band_df["B2"]
band_df["B3"] = band_df["B3"]
band_df["B4"] = band_df["B4"]
band_df["B5"] = band_df["B5"]
band_df["B6"] = band_df["B6"]
band_df["B7"] = band_df["B7"]
band_df["elevation"] = band_df["elevation"]

band_df

In [ ]:
Y = band_df.loc[:,'LULC'].values
X = band_df.loc[:,'B1':'elevation'].values

#we split the dataset into a test and training set
train_x, test_x, train_y, test_y = train_test_split(X,Y , test_size=0.3, random_state=0)
clf = LogisticRegression(solver='saga',multi_class='multinomial')


In [ ]:
clf.fit(train_x, train_y)
clf.score(test_x, test_y)

In [ ]:
#Print model parameters
print('Intercept: \n', clf.intercept_)
print('Coefficients: \n', clf.coef_)

# Same thing again Ignore

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 5)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# model1 = LogisticRegression(random_state=0, multi_class='multinomial', penalty='none', solver='newton-cg').fit(X_train, y_train)
# preds = model1.predict(X_test)

In [ ]:
# model1.score(X_test, y_test)

In [ ]:
#print the tunable parameters (They were not tuned in this example, everything kept as default)
# params = model1.get_params()
# print(params)

In [ ]:
#Print model parameters
# print('Intercept: \n', model1.intercept_)
# print('Coefficients: \n', model1.coef_)

# Random forest classifier

In [ ]:
from sklearn import ensemble

In [ ]:
# feature_names = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
# label = "landcover"
feature_names = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'NDWI', 'NDVI', 'MNDWI', 'NDBI', 'elevation']
label = 'LULC'


In [ ]:
df = band_df 
# Already divided by 20k and 10k

In [ ]:
X = df[feature_names].head(4200)
y = df[label].head(4200)

n_trees = 10 
rf = ensemble.RandomForestClassifier(n_trees).fit(X,y)

trees =  ml.rf_to_strings(rf,feature_names)

# print the first tree to see the result
# print(trees[0])
# print(trees[1])

ml.trees_to_csv(trees, "trees_all_4200.csv")

In [ ]:
rf.score(test_x, test_y)

In [ ]:
# import time
# for i in range(1000, 26000, 5000):

#   start_time = time.time()

#   X = df[feature_names].head(i)
#   y = df[label].head(i)

#   n_trees = 10 
#   rf = ensemble.RandomForestClassifier(n_trees).fit(X,y)

#   trees =  ml.rf_to_strings(rf,feature_names)

#   # print the first tree to see the result
#   # print(trees[0])
#   # print(trees[1])

#   ml.trees_to_csv(trees, "trees" + str(i) + ".csv")
#   print("Rows number: ", i , "  %s seconds" % (time.time() - start_time))

In [ ]:
# upload data to online for later use

In [ ]:
import geemap
import ee, os

os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
ee.Initialize()

In [ ]:
user_id = geemap.ee_user_id()

In [ ]:
asset_id = user_id +  "/random_forest_strings_test"
asset_id

In [ ]:
# kick off an export process so it will be saved to the ee asset
ml.export_trees_to_fc(trees,asset_id)

In [ ]:
with open("trees_all_4200.csv", "r") as file1:
    txxx = file1.read()
    
len(txxx)

In [65]:
import geemap
import ee, os

os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
ee.Initialize()
Map = geemap.Map(center=[40,-100], zoom=4)
# Add Earth Engine dataset
# Input imagery is a cloud-free Landsat 8 composite.
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')

image = ee.Algorithms.Landsat.simpleComposite(**{
  'collection': l8.filterDate('2018-01-01', '2018-12-31'),
  'asFloat': True
})

esri_lulc10 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m").median().rename('b1')
# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']

# Load training points. The numeric property 'class' stores known labels.
points = ee.FeatureCollection('GOOGLE/EE/DEMOS/demo_landcover_labels')
geemap.extract_values_to_points(points, esri_lulc10, scale = 30)

# This property of the table stores the land cover labels.
label = 'b1'

# Overlay the points on the imagery to get training.

training = esri_lulc10.addBands(image).sample({
  'scale': 30,
  'numPixels': 1000
})

# training = image.select(bands).addBands(esri_lulc10).sampleRegions(**{
#   'collection': points,
#   'properties': [label],
#   'scale': 30
# })

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

# Classify the image with the same bands used for training.
classified = image.select(bands).classify(trained)

# Display the inputs and the results.
Map.centerObject(points, 11)
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'max': 0.4}, 'image')
Map.addLayer(classified,
             {'min': 0, 'max': 2, 'palette': ['red', 'green', 'blue']},
             'classification')
Map.addLayerControl() # This line is not needed for ipyleaflet-based Map.


EEException: Invalid GeoJSON geometry.

In [59]:
Map

Map(center=[37.778253523653376, -122.37538216245599], controls=(WidgetControl(options=['position', 'transparen…